In [11]:
%load_ext jupyter_black

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.dummy import DummyClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np

# Внутренние меры кластеризации

Или как оценивать, если правильных меток не существует в природе? 



In [ ]:
features, true_labels = make_blobs(
    n_samples=200, centers=3, cluster_std=2.75, random_state=42
)

kmeans = KMeans(n_clusters=3, n_init=10)
kmeans.fit(features)

## Кстати, кто такой Rand Index ?

print("KMeans Rand Index: ", metrics.rand_score(kmeans.labels_, true_labels))

In [ ]:
## А тут вы сколько ожидаете увидеть? А если пар -- 100? 
random_labels = np.random.randint(0, 3, size=true_labels.shape)
print("Random Rand Index: ", metrics.rand_score(random_labels, true_labels))

In [ ]:
fig, (left, right) = plt.subplots(1, 2)

left.scatter(
    features[:, 0],
    features[:, 1],
    s=15,
    c=random_labels,
)

right.scatter(
    features[:, 0],
    features[:, 1],
    s=15,
    c=kmeans.labels_,
)

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score

print(f"Silhouette score")
print(f"random clustering: {silhouette_score(features, random_labels)}")
print(f"kmeans: {silhouette_score(features, kmeans.labels_)}")

print()

print(f"David-Bouldin Index")
print(f"random clustering: {davies_bouldin_score(features, random_labels)}")
print(f"kmeans: {davies_bouldin_score(features, kmeans.labels_)}")

---

# Как выбрать число кластеров


### Elbow plot
Самый простой метод, продолжающий предыдущую тему.

Какое максимальное и минимальное значения $EV$? Как они достигаются?

<img src="https://upload.wikimedia.org/wikipedia/commons/c/cd/DataClustering_ElbowCriterion.JPG" />

In [ ]:
# !pip install yellowbrick
from yellowbrick.cluster import KElbowVisualizer

visualizer = KElbowVisualizer(kmeans, k=(1, 9))

visualizer.fit(features)
visualizer.show()  # тут вместо explained variance сырые расстояния

### Silhouette analysis

Выше мы рассмотрели глобальный (усредненный) Silhouette score. Давайте теперь росмотрим на эти коэффициенты, сгруппированные по кластерам

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

# поменять число кластеров и посмотркть картинки
model = KMeans(2, random_state=42, n_init=10)
visualizer = SilhouetteVisualizer(model, colors="yellowbrick")

visualizer.fit(features)
visualizer.show()

---

## Case study: сжатие изображений

Адаптация примера из курса К.В. Воронцова.

Преобразуем изображение, приведя все значения
в интервал [0, 1]: можно использовать функцию `img_as_float` из модуля `skimage`.

In [ ]:
# ! pip install -q --upgrade scikit-image

In [ ]:
import numpy as np

import skimage
from skimage import data, io
from skimage.io import imread, imsave
from sklearn.cluster import KMeans

image = imread("Lenna.png")
print(image.shape, image.max(), image.min())

Каптинка в т.н. "полноцветном" формате, т.е. по 1 байту на каждый из трех каналов.

 **Сколько всего цветов ?**

In [ ]:
io.imshow(image)

**PSNR - peak signal-to-noise ratio**

Соотношение между максимумом возможного значения сигнала и мощностью шума, искажающего значения сигнала. Поскольку многие сигналы имеют широкий динамический диапазон, PSNR обычно измеряется в логарифмической шкале в децибелах.

PSNR наиболее часто используется для измерения уровня искажений при сжатии изображений. 

$$PSNR = 10 ~ \log_{10} \frac{MAX_{orig}^2}{MSE}$$

Довольно плохая метрика, кстати говоря; как думаете, в чем ее проблема?

In [ ]:
def psnr(X_true, X_clustered):
    
    size =...
    mse = ...
    max_2 = ...
    
    return ...


Кластеризуем, и все пиксели, отнесенные в один кластер, попробуем заполнить двумя способами: медианным и средним цветом по кластеру. Таким образом мы "сжимаем" изображение. 

In [ ]:
float_image = skimage.img_as_float(image, force_copy=True)
float_image.shape,

Переведём в набор троек RGB

In [ ]:
RGB = ...

RGB.shape, RGB

In [ ]:
import os
os.makedirs('Lenna', exist_ok=True)

clusters = [25] + list(reversed(range(2, 10)))

for n_clusters in clusters:
    
    print(n_clusters, end=" ")
    
    # группируем цвета, которые есть в изображении
    model = KMeans(n_clusters=n_clusters, verbose=False, random_state=100, n_init=10)
    model.fit(RGB)
    X = RGB.copy()
    
    # запоминаем метки кластеров
    labels = model.labels_.T

    for cluster in range(n_clusters):
        
        # по каким индексам в списке пикселей живёт этот кластер?
        ...
        
        # заполняем заполненное средним
        ...
        
    # обратно в трёхмерный вид
    im = ...
    
    print(f"Frobenius norm: {np.linalg.norm(im - float_image):3.2f}" , end=" | ")    
    print(f"PSNR: {psnr(float_image, im):3.3f}" % psnr(float_image, im))
    
    rescaled_im = (im * 255).astype(np.uint8)
    
    # сохраняем
    imsave("Lenna/" + str(n_clusters) + ".png", rescaled_im)

In [ ]:
# image = imread("Lenna/6.png")
# io.imshow(image)

In [ ]:
fig = plt.figure(figsize=(10, 10))
for i, c in enumerate(clusters):
    img = imread(f"Lenna/{c}.png")
    ax = fig.add_subplot(3, 3, i+1)
    ax.title.set_text(f'{c} clusters')
    plt.imshow(img)
plt.show()


---
## Case study 2: слова

In [28]:
# ! wget http://vectors.nlpl.eu/repository/20/182.zip
# ! bash -c "unzip -o 182.zip"

Есть три файла - `meta.json` с метаданными, `lemma.num` с частотами слов и `model.txt` собственно с эмбеддингами.

Давайте кластеризовывать слова на основе их векторных представлений.
    

In [3]:
! bash -c "tail -2 model.txt"

человечка_NOUN -0.29234073 0.03835703 0.10110392 -0.27563456 -0.15668419 0.17093626 0.16657482 0.426764 0.05025494 -0.3600553 0.41330293 -0.40945807 -0.1383572 0.18148145 -0.14004365 -0.22425707 -0.02813618 0.12828286 -0.07121982 0.08036167 -0.39346257 0.14418003 -0.112279825 0.050379105 0.65719825 0.28230706 -0.19728823 -0.22937453 -0.37179834 -0.48536462 0.03138419 -0.022372728 -0.07253032 -0.0072454247 0.054911043 0.22537158 0.105023034 0.13699752 0.25621888 -0.1780083 -0.2054968 -0.34535745 0.0049650734 0.3966938 0.08922703 -0.1604564 -0.120386116 -0.08408959 0.33405817 0.15972129 -0.01776049 -0.28450355 -0.34316248 0.08639362 -0.35194418 0.33855405 -0.31983042 0.37213647 0.2329898 0.025947459 0.0820079 -0.3286245 -0.3904638 -0.024789961 -0.21230644 0.5233826 0.37559277 -0.089096 0.0084660575 0.14825182 -0.21078822 0.32954386 0.049764797 -0.063024566 0.06718298 -0.04196897 -0.025229331 -0.35419548 0.22244026 0.08812005 -0.29118997 0.041034646 -0.049780015 0.26969633 0.15984987 0.46

In [8]:
# ! pip install nltk
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
STOPS = set(stopwords.words("russian"))
STOPS

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/arabella/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'а',
 'без',
 'более',
 'больше',
 'будет',
 'будто',
 'бы',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'вам',
 'вас',
 'вдруг',
 'ведь',
 'во',
 'вот',
 'впрочем',
 'все',
 'всегда',
 'всего',
 'всех',
 'всю',
 'вы',
 'где',
 'да',
 'даже',
 'два',
 'для',
 'до',
 'другой',
 'его',
 'ее',
 'ей',
 'ему',
 'если',
 'есть',
 'еще',
 'ж',
 'же',
 'за',
 'зачем',
 'здесь',
 'и',
 'из',
 'или',
 'им',
 'иногда',
 'их',
 'к',
 'как',
 'какая',
 'какой',
 'когда',
 'конечно',
 'кто',
 'куда',
 'ли',
 'лучше',
 'между',
 'меня',
 'мне',
 'много',
 'может',
 'можно',
 'мой',
 'моя',
 'мы',
 'на',
 'над',
 'надо',
 'наконец',
 'нас',
 'не',
 'него',
 'нее',
 'ней',
 'нельзя',
 'нет',
 'ни',
 'нибудь',
 'никогда',
 'ним',
 'них',
 'ничего',
 'но',
 'ну',
 'о',
 'об',
 'один',
 'он',
 'она',
 'они',
 'опять',
 'от',
 'перед',
 'по',
 'под',
 'после',
 'потом',
 'потому',
 'почти',
 'при',
 'про',
 'раз',
 'разве',
 'с',
 'сам',
 'свою',
 'себе',
 'себя',
 'сейчас',
 'со',
 'совсем',
 'так

In [12]:
# выбрасываем стоп-слова
freq_list = ...
len(freq_list)

for i, item in enumerate(freq_list):
    print(item)
    if i == 5:
        break
import numpy as np
from tqdm.notebook import tqdm

криминалист
грошовый
золотисто
ящерка
четвертый
комкать


In [30]:
tokens = []
raw_vectors = []

with open("model.txt", "r+", encoding="utf-8") as rf:

    # пропускаем первую строку
    next(rf)

    for line in tqdm(rf):
        # parse line, extract only vectors corresponding to frequent nouns
        ...


len(tokens), len(raw_vectors)

token2id = {t: i for i, t in enumerate(tokens)}
vectors = np.array(raw_vectors)

0it [00:00, ?it/s]

In [14]:
# нормализуем?
vectors = ...

In [15]:
vectors.shape, vectors.sum(axis=1).shape, vectors.sum(axis=1)

((16404, 300), (16404,), array([1., 1., 1., ..., 1., 1., 1.]))

In [19]:
from scipy.spatial.distance import cosine

from scipy.spatial.distance import cosine

king = raw_vectors[token2id["король"]]
queen = raw_vectors[token2id["королева"]]

man = raw_vectors[token2id["мужчина"]]
woman = raw_vectors[token2id["женщина"]]
cosine(king - man, queen - woman)

np.float64(0.23086078526958698)

In [24]:
from sklearn import cluster

mbk_means = cluster.MiniBatchKMeans(
    n_clusters=70,
    batch_size=1000,
    max_iter=10000,
    n_init=20,
    random_state=100,
    reassignment_ratio=0.1,
)
mbk_means.fit(vectors)

mbk_means.labels_

array([13, 45, 45, ..., 45, 16, 45], dtype=int32)

In [25]:
tokens2clusters = {t: c for t, c in zip(tokens, mbk_means.labels_)}
cluster2tokens = {l: [] for l in mbk_means.labels_}

In [27]:
for t, c in tokens2clusters.items():
    cluster2tokens[c].append(t)

for c in sorted(list(cluster2tokens)[:50]):
    print(f"\n>-- Cluster #{c}, {len(cluster2tokens[c])} objects.")
    print(" ".join([w for w in cluster2tokens[c][:100]]))


>-- Cluster #0, 17 objects.
мастер пояс кисть затылок убор мантия парик прадед жезл привратник рея бицепс скальп подручный чресла зашить поясной

>-- Cluster #1, 208 objects.
операция удар скорость поверхность соединение клетка спортсмен усилие восстановление обработка частота напряжение травма пластинка поворот ось нагрузка пересечение мышца комбинация электрон равновесие челюсть винт конечность ион облегчение железа сечение стопа анатомия перелом дефект сжатие клавиша сустав кальций вена артерия диаграмма форсирование симметрия хромосома твердость деформация лопасть нокаут матка наклон зубец сварка закрепление позвоночник изгиб жесткость копирование разворот позвонок поршень сцепление фиксация призма нажатие мускул испарение диафрагма ротор кровообращение чешуя отлив коррозия пигмент четкость планер запястье медитация киль визуализация бугорок выравнивание пищевод растворение бороздка антиген неподвижность кровоизлияние транс касание партнерша гортань скольжение стояние крен растяже

### Задание со звездочкой 17. Dunn index
Реализуйте Dunn Index. Будет засчитываться не абы-какая реализация, а соответствующая по стилю и оформлению реализации метрик в sklearn: обязательны докстринги, валидаторы и все такое.

То, как реализованы другие метрики, можно посмотреть [тут](https://github.com/scikit-learn/scikit-learn/blob/main/sklearn/metrics/cluster/_unsupervised.py#L195).

### Задание со звездочкой 18. Topic Modelling via naive clusterization

Мы использовали какой-то простейший метод кластеризации. На лекциях вам рассказывали еще о нескольких из них. Возьмите парочку других, прокомментировав свой выбор. Примените к тем же данным, выберете (аргументировано!) число кластеров и сравните модели между собой с помошью внутренних мер кластеризации.

In [ ]:
from sklearn.utils._param_validation import (
    validate_params,
)

@validate_params(...)
def dunn_score(...):
    """
    Compute the Dunn Index by given within-cluster distances (callable or precomputed) and
    between-cluster distances(callable or precomputed).
    ...
    
    """
    ...
    return